### Modelagem do modelo de series temporais

O modelo **SARIMAX** foi escolhido para esta análise por ser uma extensão do modelo ARIMA que incorpora a componente sazonal "S", essencial para capturar padrões periódicos na série temporal. Além disso, o SARIMAX permite a inclusão de variáveis exógenas (fatores externos), possibilitando a análise de como variáveis adicionais podem influenciar o comportametno da série principal ao longo do tempo.

Esse modelo é o modelo SARIMA, que combina os componentes ARIMA para modelar tendências não sazonais e componentes sazonais específicos para lidar com padrões que se repetem em intervalores regulares.

Parâmetros **não** sazonais

- p (autoregressive - AR): Número de defasagens da série temporal (lag).
- d (difference): número de diferenciações necessárias para tornar a série estacionária
- q (moving average - MA): Número de defasagens nos erros (resíduos)

Parâmetros sazonais

- P (sazonal AR): Número de defasagens sazonais.
- D (sazonal difference): Número de diferenciações sazonais.
- Q (sazonal MA): Número de médias móveis sazonais.
- S (sazonalidade): período da sazonalidade (ex: 12 para dados mensais com padrão anual).

O parâmetro X em SARIMAX representa a inclusão de variáveis exógenas no modelo. São variáveis externas que podem influênciar a série que está sendo modelada.


In [86]:
import pandas as pd
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [87]:
dados_modelo = pd.read_csv(r"C:\Users\eduar\OneDrive\Documentos\SandoBox\Projetos\PriceCast\Dataset\dados_preprocessados.csv",  index_col=0)
dados_modelo.head()

,populacao_pessoas,leite_uht_industria,leite_uht_supermercado,preco_leite_produtor
data,,,,
2007-02-01,188.657245,0.0,-0.007398,0.062721
2007-03-01,188.817561,0.0,0.010707,0.042416
2007-04-01,188.977878,0.0,0.030502,0.083423
2007-05-01,189.138194,0.0,0.092346,0.073984
2007-06-01,189.298511,0.0,0.151256,0.097514


In [88]:
dados_modelo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 198 entries, 2007-02-01 to 2024-12-01
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   populacao_pessoas       198 non-null    float64
 1   leite_uht_industria     198 non-null    float64
 2   leite_uht_supermercado  198 non-null    float64
 3   preco_leite_produtor    198 non-null    float64
dtypes: float64(4)
memory usage: 7.7+ KB


In [89]:
dados_modelo.index = pd.to_datetime(dados_modelo.index)

### Divisão de treino e teste

Quando se trata de séries temporais, a divisão entre dados de treinamento e teste, deve respeitar a ordem temporal dos dados. Isso significa que não se deve dividir os dados de forma aleatória, como é normalmente feito em problemas de classificação ou regressão. Pois, cada observação está dependendo do tempo. Isso significa que os dados mais recentes dependem dos valores passados para serem previstos. Se embaralhar os dados de forma aleatória irá aver uma quebra na sequência temporal  natural, o modelo pode acabar treinando com dados mais recentes e sendo testado em dados anteriores o que é irrealista para previsões futuras.


In [90]:
# Definindo a proporção de treino (80%)
dados_treinamento = int(len(dados_modelo) * 0.8)

# Dividindo o conjunto de dados, garantindo que o índice seja mantido
treino = dados_modelo.iloc[:dados_treinamento]
teste = dados_modelo.iloc[dados_treinamento:]

# Verificando se o índice está correto
print(treino.index)
print(teste.index)


DatetimeIndex(['2007-02-01', '2007-03-01', '2007-04-01', '2007-05-01',
               '2007-06-01', '2007-07-01', '2007-08-01', '2007-09-01',
               '2008-02-01', '2008-03-01',
               ...
               '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01',
               '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
               '2021-07-01', '2021-08-01'],
              dtype='datetime64[ns]', name='data', length=158, freq=None)
DatetimeIndex(['2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
               '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01',
               '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01',
               '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01',
               '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01',
               '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01',
               '2024-01-01', '2024-02-0

In [91]:
# Separando a variável alvo
y_train = treino['preco_leite_produtor']
y_test = teste['preco_leite_produtor']

X_train = treino.drop(columns='preco_leite_produtor')
X_test = teste.drop(columns='preco_leite_produtor')


In [92]:
# Convertendo os índices para datetime
X_train.index = pd.to_datetime(X_train.index)
y_train.index = pd.to_datetime(y_train.index)
X_test.index = pd.to_datetime(X_test.index)
y_test.index = pd.to_datetime(y_test.index)

### Otimização dos Hiperparâmetros

O algoritmo auto_arima, realiza a otimização automática dos parâmetros do modelo SARIMAX, testando múltiplcas combinações de (p, d, q) e (P, D, Q, S). Ele aplica testes estatísticos como o teste de estacionariedade Dickey Fuller Aumentado (ADF) o que já foi aplicado na etapa anterior. Além disso, avalia o desempenho dos modelos com base em critérior como AIC e BIC. O modelo resultadte é o que apresenta o melhor equilíbrio entre ajuste e complexidade, garantindo maior capacidade de generalização.


- stepwise = true: Usa busca sequencial inteligente para reduzir o tempo de busca por melhores parâmetros.
- trace = true: Exibe no console os passos da busca e os resultados de AIC de cada modelo testado

In [ ]:
modelo_optimization = auto_arima(y_train, exogenous = X_train, seasonal = True, m = 12, stepwise = True, trace = True, suppress_warnings = True)

 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=-476.158, Time=0.05 sec
 ARIMA(0,0,0)(0,0,1)[12] intercept   : AIC=-474.427, Time=0.15 sec
 ARIMA(0,0,0)(0,0,2)[12] intercept   : AIC=-477.204, Time=0.51 sec
 ARIMA(0,0,0)(1,0,0)[12] intercept   : AIC=-474.513, Time=0.10 sec
 ARIMA(0,0,0)(1,0,1)[12] intercept   : AIC=-474.808, Time=0.29 sec
 ARIMA(0,0,0)(1,0,2)[12] intercept   : AIC=-475.737, Time=0.82 sec
 ARIMA(0,0,0)(2,0,0)[12] intercept   : AIC=-477.345, Time=0.39 sec
 ARIMA(0,0,0)(2,0,1)[12] intercept   : AIC=-475.562, Time=0.87 sec
 ARIMA(0,0,0)(2,0,2)[12] intercept   : AIC=inf, Time=1.23 sec
 ARIMA(0,0,1)(0,0,0)[12] intercept   : AIC=-536.485, Time=0.05 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=-534.496, Time=0.21 sec
 ARIMA(0,0,1)(0,0,2)[12] intercept   : AIC=-533.867, Time=0.82 sec
 ARIMA(0,0,1)(1,0,0)[12] intercept   : AIC=-534.498, Time=0.13 sec
 ARIMA(0,0,1)(1,0,1)[12] intercept   : AIC=inf, Time=0.42 sec
 ARIMA(0,0,1)(1,0,2)[12] intercept   : AIC=-532.857, Time=1.27 sec
 ARIM

Analise

Com base no resultado do algoritmo. Ele acabou decidindo que não era necessário modelar a parte sazonal.

In [94]:
# Treinando o modelo final com os dados de treino
modelo_final = modelo_optimization
modelo_final.fit(y_train, exogenous=X_train)

ARIMA(order=(2, 0, 2), scoring_args={}, seasonal_order=(0, 0, 0, 12),
      suppress_warnings=True)

In [100]:
# Prevendo com os dados de teste
y_pred = modelo_final.predict(n_periods=len(y_test), exogenous=X_test)

c:\Users\eduar\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\eduar\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
